<a href="https://colab.research.google.com/github/Eunhye1109/Practical-Project/blob/EH/250728_OpenAI_2%EC%B0%A8_test_%EC%B5%9C%EC%9D%80%ED%98%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1차 OpenAI 자연어처리 성공 코드

In [ ]:
# ✅ 0. 필수 설치
!pip install --quiet openai PyMuPDF

# ✅ 1. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 2. 임포트
import os, re, unicodedata
import fitz
import openai
import torch
from sentence_transformers import SentenceTransformer, util

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 70.1 MB/s eta 0:00:00
Mounted at /content/drive


In [ ]:
# ✅ 3. OpenAI 최신 방식 (openai>=1.0.0)
client = openai.OpenAI(api_key="키를 입력하세요")  # ← 본인의 키 입력

# ✅ 4. 사용자 입력
company = input("요약할 기업명을 입력하세요: ").strip()
company = unicodedata.normalize("NFC", company)

# ✅ 5. PDF 폴더 직접 지정
search_folder = "/content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서"
assert os.path.exists(search_folder), "❌ 지정한 PDF 폴더 경로가 존재하지 않습니다."

# ✅ 6. PDF 파일 탐색
files = os.listdir(search_folder)
matched = [f for f in files if company in unicodedata.normalize("NFC", f) and f.lower().endswith(".pdf")]
if not matched:
    raise FileNotFoundError(f"❌ '{company}' 관련 PDF 파일을 찾을 수 없습니다.")
pdf_path = os.path.join(search_folder, matched[0])
print("✅ 찾은 PDF:", pdf_path)

# ✅ 7. PDF → 텍스트 추출
with fitz.open(pdf_path) as doc:
    raw_text = "".join([page.get_text() for page in doc])

# ✅ 8. 텍스트 전처리 및 문장 필터링
text = re.sub(r'\[\d+\]', '', raw_text)
text = re.sub(r'\s+', ' ', text)
sentences = re.split(r'(?<=[.?!])\s+', text)

def is_valid(s):
    s = s.strip()
    return len(s) > 10 and sum(c.isdigit() for c in s)/len(s) < 0.2 and len(re.findall(r'[가-힣a-zA-Z]', s)) >= 5

filtered = [s for s in sentences if is_valid(s)]
print(f"📚 유효 문장 수: {len(filtered)}")

# ✅ 9. 요약 주제 정의
topics = {
    "📊 재무 요약": "최근 3개년 손익계산서를 기준으로 매출액, 영업이익, 당기순이익 등을 요약하세요.",
    "🛡️ 리스크 및 부채": "부채, 유동성 리스크, 지급보증, 담보 제공 등과 관련된 내용을 요약하세요.",
    "🚀 사업 전략": "신규 사업, 투자 계획, 성장 전략 등을 요약하세요.",
    "🔬 기술 및 R&D": "연구개발 현황, 기술 파이프라인, 기술 제휴 등을 요약하세요.",
    "🌐 글로벌 진출": "해외 시장 전략, 수출 확대, 해외 법인 관련 내용을 요약하세요.",
    }

# ✅ 10. GPT 요약 함수
def gpt_summarize_section(text, topic_instruction):
    prompt = f"""다음은 한국 기업의 사업보고서 일부입니다.

요약 주제:
{topic_instruction}

아래 텍스트를 기반으로 한 문단(3~5문장)으로 요약해 주세요.

<본문>
{text[:2000]}"""  # 토큰 초과 방지

    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.4,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"❌ 요약 실패: {e}"

# ✅ 11. 요약 실행
text_sample = " ".join(filtered[:300])
results = {}
for sec, inst in topics.items():
    results[sec] = gpt_summarize_section(text_sample, inst)

# ✅ 12. 결과 출력 및 저장
output_txt = ""
for sec, summ in results.items():
    output_txt += f"\n\n## {sec}\n{summ}"

output_path = pdf_path.replace(".pdf", "_GPT요약.txt")
with open(output_path, "w", encoding="utf-8") as f:
    f.write(output_txt.strip())

print("✅ GPT 요약 완료! 저장 위치:", output_path)
print(output_txt)


요약할 기업명을 입력하세요: 카카오
✅ 찾은 PDF: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/카카오 [기재정정]사업보고서 (2024.12).pdf
📚 유효 문장 수: 3499
✅ GPT 요약 완료! 저장 위치: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/카카오 [기재정정]사업보고서 (2024.12)_GPT요약.txt


## 📊 재무 요약
최근 3개년 동안의 손익계산서를 기준으로 살펴보면, 이 한국 기업의 매출액은 꾸준히 증가하고 있는 추세를 보였습니다. 영업이익 또한 매출액 증가에 따라 상승세를 유지하며, 기업의 운영 효율성이 개선되고 있음을 나타냅니다. 당기순이익 역시 긍정적인 흐름을 보이며, 기업의 전반적인 재무 건전성이 강화되고 있음을 시사합니다. 이러한 재무 성과는 기업의 지속 가능한 성장 가능성을 뒷받침하고 있습니다.

## 🛡️ 리스크 및 부채
이 기업의 사업보고서에서는 부채와 관련하여 기업의 현재 부채 수준과 상환 계획을 명시하고 있습니다. 유동성 리스크는 단기적인 자금 조달의 어려움과 관련된 위험 요소로 언급되며, 이를 완화하기 위한 대책이 포함되어 있습니다. 또한, 기업은 특정 거래에 대해 지급보증을 제공하고 있으며, 이에 따른 재무적 영향을 평가하고 있습니다. 마지막으로, 담보 제공과 관련하여 기업이 보유한 자산을 담보로 제공한 내역과 그에 따른 책임이 설명되어 있습니다.

## 🚀 사업 전략
이 기업은 제조서비스업 분야에서의 성장을 위해 신규 사업을 추진하고 있으며, 이를 통해 시장 점유율을 확대하고자 합니다. 또한, 최신 기술 도입과 생산 설비 확장을 위한 투자를 계획하고 있어, 장기적

## 2차 자연어처리 성공 코드

In [ ]:
# ✅ 13. 2차 요약 시작: 1차 요약 기반 Executive Summary 생성

# 2차 요약 주제 정의 (원하면 자유롭게 커스터마이즈 가능)
second_topics = {
    "📊 재무 요약": "최근 3개년 손익계산서를 기준으로 매출액, 영업이익, 당기순이익 등을 요약하세요.",
    "🛡️ 리스크 및 부채": "부채, 유동성 리스크, 지급보증, 담보 제공 등과 관련된 내용을 요약하세요.",
    "🚀 사업 전략": "신규 사업, 투자 계획, 성장 전략 등을 요약하세요.",
    "🔬 기술 및 R&D": "연구개발 현황, 기술 파이프라인, 기술 제휴 등을 요약하세요.",
    "🌐 글로벌 진출": "해외 시장 전략, 수출 확대, 해외 법인 관련 내용을 요약하세요.",
}

# GPT 호출 함수
def gpt_second_pass(first_summary_text: str, topics: dict) -> str:
    output_lines = []
    for sec, inst in topics.items():
        prompt = f"""당신은 금융 애널리스트입니다. 다음은 기업의 1차 요약본입니다.

[요약 주제]
{inst}

[1차 요약]
{first_summary_text}

위 내용을 기반으로, 한 문장 내로, 그리고 중학생도 이해할 수 있을 만큼 쉬운 말로, 그러나 명확하고 구체적인 수치와 사례 기반으로 요약하세요.
'이 한국 기업은', 이라는 말 대신 '이 기업은' 으로 대체하세요.

"""
        try:
            resp = client.chat.completions.create(
                model="gpt-4o",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.3,
            )
            content = resp.choices[0].message.content.strip()
        except Exception as e:
            content = f"❌ 오류 발생: {e}"
        output_lines.append(f"\n\n## {sec}\n{content}")
    return "# 2차 요약 결과\n" + "".join(output_lines).strip()


# ✅ 14. 1차 요약 텍스트 불러오기
with open(output_path, "r", encoding="utf-8") as f:
    first_summary_text = f.read()

# ✅ 15. 2차 요약 생성
second_summary_text = gpt_second_pass(first_summary_text, second_topics)

# ✅ 16. 저장 경로 정의 및 저장
second_output_path = output_path.replace("_GPT요약.txt", "_GPT2차요약.md")
with open(second_output_path, "w", encoding="utf-8") as f:
    f.write(second_summary_text)

# ✅ 17. 완료 출력
print("✅ 2차 요약 완료! 저장 위치:", second_output_path)
print(second_summary_text)


✅ 2차 요약 완료! 저장 위치: /content/drive/MyDrive/2. 취업 부업/7. SW개발자과정/3. 실전프로젝트/[실전] 재무 관련 데이터_호성 팀/PDF 사업 보고서/카카오 [기재정정]사업보고서 (2024.12)_GPT2차요약.md
# 2차 요약 결과
## 📊 재무 요약
이 기업은 지난 3년 동안 매출과 이익이 꾸준히 증가하고 있으며, 부채 관리와 기술 개발에 집중하면서 글로벌 시장에서도 입지를 넓히고 있습니다.

## 🛡️ 리스크 및 부채
이 기업은 부채를 관리하고 유동성 리스크를 줄이기 위해 상환 계획을 세우고 있으며, 특정 거래에 대해 지급보증을 제공하고 보유 자산을 담보로 활용하고 있습니다.

## 🚀 사업 전략
이 기업은 매출과 이익이 꾸준히 증가하고 있으며, 새로운 사업과 기술 개발에 투자하여 글로벌 시장에서 더 많은 제품을 팔고, 해외에 더 많은 지사를 세우려는 계획을 가지고 있습니다.

## 🔬 기술 및 R&D
이 기업은 다양한 기술 개발과 외부 협력을 통해 혁신적인 제품을 만들고 있으며, 이를 통해 시장에서 더 많은 경쟁력을 얻고 있습니다.

## 🌐 글로벌 진출
이 기업은 해외 시장에서 경쟁력을 높이기 위해 맞춤형 제품과 서비스를 제공하고, 해외 법인을 설립하여 수출을 늘리고 있습니다.
